In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch import nn
import torchviz
# import dnabert
# import vit
import scipy.io
from matplotlib import pyplot as plt
import numpy as np
import dataset
import dnabert
from importlib import reload

/home/amintehrani/.pyenv/versions/3.12.8/envs/torchgeo/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
MAT_FILE_PATH = '/mnt/sdb4/insect_dataset.mat'

mat = scipy.io.loadmat(MAT_FILE_PATH)
mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'all_images', 'all_dnas', 'all_labels', 'all_dnas_norepeat', 'all_dna_labels_norepeat', 'all_boldids', 'train_loc', 'val_seen_loc', 'val_unseen_loc', 'test_seen_loc', 'test_unseen_loc', 'species2genus', 'described_species_labels_train', 'described_species_labels_trainval', 'all_dna_features_cnn_original', 'all_image_features_resnet', 'all_image_features_gan', 'all_dna_features_cnn_new', 'all_string_dnas'])

In [ ]:
# dna_dataset = DNADataset(mat['all_string_dnas'], mat['all_labels'], dnabert.tokenizer)

# dnabert.model(**dnabert.tokenizer(dna[:10].tolist(), return_tensors='pt', padding=True, truncation=True, max_length=512, return_attention_mask=True)).last_hidden_state[:,0,:].shape

In [ ]:
# reload(dnabert)
# reload(dataset)
# fine_tune_res = dnabert.finetune(mat, 1050)

In [5]:
from sklearn.decomposition import PCA

pca = PCA(n_components=512)
all_dna_features_cnn_pca = pca.fit_transform(mat['all_dna_features_cnn_new'])


In [6]:
pca = PCA(n_components=512)
all_image_features_gan_pca = pca.fit_transform(mat['all_image_features_gan'])

In [32]:
mat['all_boldids']

array(['FBCOA785-10  ', 'FBCOB090-10  ', 'FBCOJ742-13  ', ...,
       'SDP902190-20 ', 'SDP902222-20 ', 'SDP902158-20 '],
      shape=(32424,), dtype='<U13')

In [20]:
import models
reload(models)
from models import AttentionFusion, Predictor, Decoder
device = torch.device('cpu')
fusion = AttentionFusion(512, 512, 512).to(device)
decoder = Decoder(512, 1050).to(device)
predictor = Predictor(fusion, decoder).to(device)
predictor.fit(all_dna_features_cnn_pca, 
              all_image_features_gan_pca, 
              mat['all_labels'].squeeze(), 
              mat['val_seen_loc'].squeeze(), 
              mat['train_loc'].squeeze(), 
              500, 
              lr=0.0005,
              device=device)

{'train_loss': [6.956546783447266,
  6.956544399261475,
  6.956541538238525,
  6.956539154052734],
 'val_loss': [6.956546783447266,
  6.956542491912842,
  6.956540107727051,
  6.95653772354126],
 'val_accuracy': [0.0, 0.0, 0.0, 0.0012368583797155227],
 'val_ari': [0.06426883611852384,
  0.06573385485768057,
  0.0660427554166934,
  0.06438161225139968],
 'val_nmi': [0.656810221003117,
  0.6579820356582857,
  0.6574414865281533,
  0.6545028017322444]}

In [26]:
all_dna_features_cnn_pca.shape

(32424, 768)

In [ ]:
np.unique(mat['all_labels'])

array([   1,    2,    3, ..., 1048, 1049, 1050], shape=(1050,))

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DNAImageDecoder(nn.Module):
    def __init__(self, N_dna, N_image, d_model=128, num_heads=4, num_classes=10):
        super().__init__()
        
        # Project DNA and image embeddings into same space
        self.dna_proj = nn.Linear(N_dna, d_model)
        self.img_proj = nn.Linear(N_image, d_model)
        
        # Self-attention mechanism
        self.attn = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads, batch_first=True)
        
        # Feed-forward layer after attention
        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
            nn.Linear(d_model, num_classes)
        )
        
    def forward(self, dna_emb, img_emb):
        """
        dna_emb: (batch_size, N_dna)
        img_emb: (batch_size, N_image)
        """
        # Project to same dimension
        dna_token = self.dna_proj(dna_emb).unsqueeze(1)  # (batch, 1, d_model)
        img_token = self.img_proj(img_emb).unsqueeze(1)  # (batch, 1, d_model)
        
        # Sequence: [DNA, Image]
        seq = torch.cat([dna_token, img_token], dim=1)  # (batch, 2, d_model)
        
        # Self-attention
        attn_out, _ = self.attn(seq, seq, seq)  # (batch, 2, d_model)
        
        # Pooling — use first token (DNA) or mean-pool
        pooled = attn_out.mean(dim=1)  # (batch, d_model)
        
        # Classification
        logits = self.ffn(pooled)  # (batch, num_classes)
        return logits


In [43]:
ma = nn.MultiheadAttention(768, 4)
attn_output, attn_output_weights = (ma(torch.rand(1, 1, 768), torch.rand(1, 1, 768), torch.rand(1, 1, 768)))


In [47]:
attn_output_weights

tensor([[[1.]]], grad_fn=<MeanBackward1>)

In [ ]:
dna = "ACGTAGCATCGGATCTATCTATCGACACTTGGTTATCGATCTACGAGCATCTCGTTAGC"
dna_emb = dnabert.model(**dnabert.tokenizer(dna, return_tensors='pt',))


odict_keys(['last_hidden_state', 'pooler_output'])

In [21]:
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import torch

# Load pretrained ViT model
model_name = "google/vit-base-patch16-224"
imageprocessor = AutoImageProcessor.from_pretrained(model_name)
vitmodel = AutoModel.from_pretrained(model_name)
vitmodel.eval()

# Put model on GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vitmodel.to(device)

def get_vit_embedding(image_np, model_name="google/vit-base-patch16-224", device = torch.device("cuda" if torch.cuda.is_available() else "cpu")):
    inputs = imageprocessor(images=image_np, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = vitmodel(**inputs)
    
    return outputs.last_hidden_state[:, 0]  # CLS token


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
dna_emb.pooler_output.shape

torch.Size([1, 768])